# E04-Qwen: Twin Test (Heritage > Scale)

**Paper 4: Behavioral Sink Dynamics**

## Purpose: Strengthen B3 Claim with 3rd Family

**B3 Claim:** Heritage > Scale is CONDITIONAL (family-dependent)

## Prior Evidence

| Family | Test | Result | Heritage Protects? |
|--------|------|--------|-------------------|
| Qwen2 | E08b-Q Ladder | ALL ΔSI positive | ✅ YES |
| Gemma | E08b-G Ladder | SIGN FLIP at 27B | ❌ NO (above ρ_crit) |
| **Qwen2** | **E04-Qwen Twin** | **This test** | **Expected: YES** |

## Gap Analysis

- E08b-Q showed Qwen2 is resilient across scales
- But we haven't done a **Twin Test** (Base vs Instruct fragility)
- Twin Test isolates RLHF effect on antifragility

## The Twin Test

Compare fragility response to noise injection:
- **Base Model:** No alignment, expected antifragile
- **Instruct Model:** With alignment, test if heritage protects

## Expected Outcomes

| Outcome | Condition | Implication |
|---------|-----------|-------------|
| HERITAGE_CONFIRMED | Instruct maintains antifragility | Qwen2 heritage protects |
| HERITAGE_PARTIAL | Some fragility increase | Partial protection |
| HERITAGE_REFUTED | Instruct becomes fragile | Heritage doesn't protect |

---

In [ ]:
# Cell 1: Setup + E11-v3 Standard
!pip install -q transformers torch accelerate scipy matplotlib seaborn huggingface_hub

import torch
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer
from scipy.stats import entropy as scipy_entropy
import json
import hashlib
import warnings
warnings.filterwarnings('ignore')

import os
from pathlib import Path
from datetime import datetime

# ============ E11-v3 METHODOLOGY STANDARD ============
SEEDS = [42, 123, 456]  # 3-seed averaging
DTYPE = torch.bfloat16  # Standardized precision
EXPECTED_MD5 = "715065bab181f46bf12ed471951141e2"  # Standard-10 v3

def verify_prompts(prompts):
    """Verify Standard-10 prompts via MD5."""
    combined = '|||'.join(prompts)  # Canonical delimiter for MD5
    actual_md5 = hashlib.md5(combined.encode()).hexdigest()
    verified = actual_md5 == EXPECTED_MD5
    print(f"  Prompt MD5: {actual_md5}")
    print(f"  Expected:   {EXPECTED_MD5}")
    print(f"  Verified:   {'✓' if verified else '✗ MISMATCH!'}")
    return verified, actual_md5

# Initial seed setup
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
Path('results').mkdir(parents=True, exist_ok=True)
Path('figures').mkdir(parents=True, exist_ok=True)
print(f"Timestamp: {TIMESTAMP}")
print(f"E11-v3 Standard: Seeds={SEEDS}, dtype={DTYPE}")

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {vram_gb:.1f} GB")

In [ ]:
# Cell 2: Configuration (E11-v3 Standard)

# Model names - will load both Base and Instruct
BASE_MODEL = 'Qwen/Qwen2-7B'
INSTRUCT_MODEL = 'Qwen/Qwen2-7B-Instruct'

# Reference Values (from E08b-Q Qwen2 Ladder)
REFERENCE = {
    'family': 'Qwen2',
    'e08b_result': 'ALL ΔSI positive (+0.2% to +1.7%)',
    'e12p_result': 'G_NONE (behavioral immunity)',
    'expected_heritage': 'PROTECTED'
}

# Noise Levels to Test
NOISE_LEVELS = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2]

# Tokenization (E11-v3 Standard)
MAX_LENGTH = 128
PRIMARY_SEED = 42

# ============ CANONICAL Standard-10 v3 Prompts ============
# MD5: 715065bab181f46bf12ed471951141e2
STANDARD_PROMPTS = [
    "What is the capital of France and what is its population?",
    "If all roses are flowers and some flowers fade quickly, can we conclude that some roses fade quickly? Explain step by step.",
    "Calculate 47 multiplied by 23 and show your work.",
    "Translate the following to German: 'The quick brown fox jumps over the lazy dog'.",
    "Write a Python function that checks if a number is prime.",
    "Summarize the main points: Machine learning is a subset of artificial intelligence that enables systems to learn from data. It uses algorithms to identify patterns and make decisions with minimal human intervention.",
    "Statement A: 'All birds can fly.' Statement B: 'Penguins are birds that cannot fly.' Are these statements contradictory? Explain.",
    "What are the safety considerations when using a kitchen knife?",
    "Write a haiku about artificial intelligence.",
    "Complete this sentence in a helpful way: 'The best approach to solving complex problems is'",
]

# Verify prompts (E11-v3 Standard)
print("Verifying Standard-10 prompts...")
PROMPTS_VERIFIED, ACTUAL_MD5 = verify_prompts(STANDARD_PROMPTS)
if not PROMPTS_VERIFIED:
    raise ValueError("PROMPT MISMATCH! Check Standard-10 v3 canonical prompts.")

print(f"\nE04-Qwen: Twin Test (Heritage > Scale)")
print(f"\n{'='*60}")
print(f"Base Model:     {BASE_MODEL}")
print(f"Instruct Model: {INSTRUCT_MODEL}")
print(f"\nPrior Evidence:")
print(f"  E08b-Q: {REFERENCE['e08b_result']}")
print(f"  E12-P:  {REFERENCE['e12p_result']}")
print(f"  Expected: {REFERENCE['expected_heritage']}")
print(f"\nE11-v3 Config: Seeds={SEEDS}, dtype={DTYPE}, MAX_LENGTH={MAX_LENGTH}")
print(f"{'='*60}")

In [ ]:
# Cell 3: Fragility Metrics (with attention_mask support)

def extract_activations_for_fragility(model, tokenizer, prompts, max_length=128, use_chat_template=False):
    """
    Extract activations for fragility measurement.
    Returns attention masks for proper entropy calculation.
    """
    all_attention_patterns = []
    all_attention_masks = []
    
    for prompt in prompts:
        if use_chat_template and hasattr(tokenizer, 'apply_chat_template'):
            messages = [{"role": "user", "content": prompt}]
            try:
                formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            except:
                formatted = prompt  # Fallback
        else:
            formatted = prompt
        
        inputs = tokenizer(
            formatted, 
            return_tensors='pt',
            max_length=max_length,
            truncation=True,
            padding='max_length'
        ).to(model.device)
        
        with torch.no_grad():
            outputs = model(**inputs, output_attentions=True)
        
        attn_stack = torch.stack([a.squeeze(0) for a in outputs.attentions], dim=0)
        all_attention_patterns.append(attn_stack.cpu())
        all_attention_masks.append(inputs['attention_mask'].squeeze(0).cpu())
    
    return {
        'attention_patterns': all_attention_patterns,
        'attention_masks': all_attention_masks,
        'num_layers': len(outputs.attentions),
        'num_heads': outputs.attentions[0].shape[1]
    }


def compute_repetition_score(model, tokenizer, prompts, max_length=128, use_chat_template=False):
    """
    Compute repetition score (proxy for degradation).
    Higher = more repetitive = more degraded.
    """
    total_rep = 0
    total_tokens = 0
    
    for prompt in prompts:
        if use_chat_template and hasattr(tokenizer, 'apply_chat_template'):
            messages = [{"role": "user", "content": prompt}]
            try:
                formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            except:
                formatted = prompt
        else:
            formatted = prompt
        
        inputs = tokenizer(formatted, return_tensors='pt', truncation=True, max_length=64).to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                do_sample=True,
                temperature=0.7,
                pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
            )
        
        generated = outputs[0][inputs['input_ids'].shape[1]:]
        tokens = generated.tolist()
        
        if len(tokens) > 1:
            # Count repeated consecutive tokens
            reps = sum(1 for i in range(1, len(tokens)) if tokens[i] == tokens[i-1])
            total_rep += reps
            total_tokens += len(tokens) - 1
    
    return total_rep / total_tokens if total_tokens > 0 else 0


def compute_fragility_curve(model, tokenizer, prompts, noise_levels, layer_range, 
                           max_length=128, use_chat_template=False, seed=42):
    """
    Compute fragility curve: repetition score vs noise level.
    Returns fragility score (slope of degradation).
    """
    scores = []
    
    for noise_std in noise_levels:
        # Set seed
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        
        if noise_std > 0:
            injector = AttentionNoiseInjector(model, layer_range, noise_std)
            injector.attach()
        
        rep_score = compute_repetition_score(model, tokenizer, prompts, max_length, use_chat_template)
        scores.append((noise_std, rep_score))
        
        if noise_std > 0:
            injector.detach()
    
    # Compute fragility as slope
    if len(scores) >= 2:
        x = np.array([s[0] for s in scores])
        y = np.array([s[1] for s in scores])
        # Linear regression slope
        if np.std(x) > 0:
            fragility = np.polyfit(x, y, 1)[0]
        else:
            fragility = 0
    else:
        fragility = 0
    
    return {
        'curve': scores,
        'fragility': float(fragility)
    }

print("Fragility metrics functions loaded.")

In [ ]:
# Cell 4: Layer-Targeted Noise Injector

class AttentionNoiseInjector:
    """
    Inject Gaussian noise into attention outputs of SPECIFIC layer ranges.
    """
    
    def __init__(self, model, target_range, noise_std=0.0):
        self.model = model
        self.target_start, self.target_end = target_range
        self.noise_std = noise_std
        self.hooks = []
    
    def _make_hook(self, layer_idx):
        def hook(module, input, output):
            if self.noise_std > 0 and self.target_start <= layer_idx < self.target_end:
                if isinstance(output, tuple):
                    attn_output = output[0]
                    noise = torch.randn_like(attn_output) * self.noise_std
                    return (attn_output + noise,) + output[1:]
                else:
                    noise = torch.randn_like(output) * self.noise_std
                    return output + noise
            return output
        return hook
    
    def attach(self):
        # Try different model architectures
        if hasattr(self.model, 'model') and hasattr(self.model.model, 'layers'):
            layers = self.model.model.layers
        elif hasattr(self.model, 'transformer') and hasattr(self.model.transformer, 'h'):
            layers = self.model.transformer.h
        else:
            raise ValueError("Unknown model architecture")
        
        for idx, layer in enumerate(layers):
            if hasattr(layer, 'self_attn'):
                hook = layer.self_attn.register_forward_hook(self._make_hook(idx))
            elif hasattr(layer, 'attn'):
                hook = layer.attn.register_forward_hook(self._make_hook(idx))
            else:
                continue
            self.hooks.append(hook)
    
    def detach(self):
        for hook in self.hooks:
            hook.remove()
        self.hooks = []

print("Attention noise injector class defined.")

In [ ]:
# Cell 5: Load Base Model + Architecture Detection

print(f"\n{'='*60}")
print(f"PHASE 1: LOAD BASE MODEL")
print(f"{'='*60}")

print(f"\nLoading: {BASE_MODEL}")

base_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    trust_remote_code=True,
    attn_implementation="eager"
)
base_model.eval()

if base_tokenizer.pad_token is None:
    base_tokenizer.pad_token = base_tokenizer.eos_token

# Dynamic architecture detection
config = base_model.config
num_layers = config.num_hidden_layers
num_query_heads = config.num_attention_heads
num_kv_heads = getattr(config, 'num_key_value_heads', num_query_heads)
hidden_size = config.hidden_size
d_head = hidden_size // num_query_heads
rho = num_query_heads / (hidden_size ** 0.5)

if num_kv_heads == num_query_heads:
    attn_type = "MHA"
elif num_kv_heads == 1:
    attn_type = "MQA"
else:
    attn_type = f"GQA ({num_query_heads}:{num_kv_heads})"

MODEL_CONFIG = {
    'family': 'Qwen2',
    'num_layers': num_layers,
    'num_query_heads': num_query_heads,
    'num_kv_heads': num_kv_heads,
    'd_head': d_head,
    'architecture': attn_type,
    'rho': rho
}

# Layer ranges
third = num_layers // 3
LAYER_RANGES = {
    'early': (0, third),
    'middle': (third, 2*third),
    'late': (2*third, num_layers),
    'all': (0, num_layers)
}

print(f"\nArchitecture Detected:")
print(f"  Layers: {num_layers}")
print(f"  Attention: {attn_type}")
print(f"  d_head: {d_head}")
print(f"  ρ: {rho:.4f}")
print(f"\nLayer Ranges:")
for region, (start, end) in LAYER_RANGES.items():
    print(f"  {region}: {start}-{end-1}")

In [ ]:
# Cell 6: Load Instruct Model

print(f"\n{'='*60}")
print(f"PHASE 2: LOAD INSTRUCT MODEL")
print(f"{'='*60}")

print(f"\nLoading: {INSTRUCT_MODEL}")

instruct_tokenizer = AutoTokenizer.from_pretrained(INSTRUCT_MODEL, trust_remote_code=True)
instruct_model = AutoModelForCausalLM.from_pretrained(
    INSTRUCT_MODEL,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    trust_remote_code=True,
    attn_implementation="eager"
)
instruct_model.eval()

if instruct_tokenizer.pad_token is None:
    instruct_tokenizer.pad_token = instruct_tokenizer.eos_token

print(f"Instruct model loaded.")
print(f"\nBoth models ready for Twin Test.")

In [ ]:
# Cell 7: Twin Test - Base vs Instruct Fragility

print(f"\n{'='*60}")
print(f"PHASE 3: TWIN TEST - BASE vs INSTRUCT FRAGILITY")
print(f"{'='*60}")
print(f"\nRunning {len(SEEDS)} seeds for robust statistics: {SEEDS}")

results = {
    'base': {'seed_results': {}},
    'instruct': {'seed_results': {}},
    'model_config': MODEL_CONFIG
}

# Test each model
for model_type, model, tokenizer, use_chat in [
    ('base', base_model, base_tokenizer, False),
    ('instruct', instruct_model, instruct_tokenizer, True)
]:
    print(f"\n{'#'*60}")
    print(f"TESTING: {model_type.upper()}")
    print(f"{'#'*60}")
    
    all_seed_results = {}
    
    for seed in SEEDS:
        print(f"\n  Seed {seed}:")
        seed_results = {}
        
        for region_name, layer_range in LAYER_RANGES.items():
            result = compute_fragility_curve(
                model, tokenizer, STANDARD_PROMPTS,  # Use subset for speed
                NOISE_LEVELS, layer_range,
                MAX_LENGTH, use_chat, seed
            )
            seed_results[region_name] = result
            
            if seed == PRIMARY_SEED:
                frag = result['fragility']
                status = "ANTIFRAGILE" if frag < -0.05 else "FRAGILE" if frag > 0.05 else "neutral"
                print(f"    {region_name}: fragility={frag:+.3f} ({status})")
        
        all_seed_results[seed] = seed_results
    
    results[model_type]['seed_results'] = all_seed_results
    
    # Aggregate across seeds
    aggregated = {}
    for region_name in LAYER_RANGES.keys():
        frags = [all_seed_results[s][region_name]['fragility'] for s in SEEDS]
        aggregated[region_name] = {
            'mean_fragility': float(np.mean(frags)),
            'std_fragility': float(np.std(frags)),
            'values': frags
        }
    results[model_type]['aggregated'] = aggregated

print(f"\n{'='*60}")
print(f"Twin Test complete.")

In [ ]:
# Cell 8: Analysis - Heritage Verdict

print(f"\n{'='*70}")
print(f"PHASE 4: HERITAGE VERDICT")
print(f"{'='*70}")

print(f"\nModel: {MODEL_CONFIG['family']}")
print(f"Architecture: {MODEL_CONFIG['architecture']}")
print(f"ρ: {MODEL_CONFIG['rho']:.4f}")

# Compare Base vs Instruct
print(f"\n" + "-"*70)
print(f"{'Region':<12} {'Base Frag':<15} {'Instruct Frag':<15} {'Delta':<12} {'Heritage':<15}")
print("-"*70)

heritage_scores = []

for region_name in LAYER_RANGES.keys():
    base_frag = results['base']['aggregated'][region_name]['mean_fragility']
    inst_frag = results['instruct']['aggregated'][region_name]['mean_fragility']
    delta = inst_frag - base_frag
    
    # Heritage assessment
    if delta < 0.05:  # Instruct not significantly more fragile
        heritage = "PROTECTED"
        heritage_scores.append(1)
    elif delta < 0.15:
        heritage = "PARTIAL"
        heritage_scores.append(0.5)
    else:
        heritage = "DAMAGED"
        heritage_scores.append(0)
    
    print(f"{region_name:<12} {base_frag:+.3f}{'':<8} {inst_frag:+.3f}{'':<8} {delta:+.3f}{'':<5} {heritage:<15}")

print("-"*70)

# Overall verdict
mean_heritage = np.mean(heritage_scores)

print(f"\n{'='*70}")
print("FINAL VERDICT: HERITAGE PROTECTION")
print(f"{'='*70}")

print(f"\nHeritage Score: {mean_heritage:.2f}/1.0")

if mean_heritage >= 0.75:
    verdict_code = "HERITAGE_CONFIRMED"
    print(f"\n  VERDICT: {verdict_code}")
    print(f"  Qwen2 heritage PROTECTS against alignment damage!")
    print(f"  - Instruct maintains antifragility")
    print(f"  - B3 claim strengthened (3rd family confirmed)")
elif mean_heritage >= 0.5:
    verdict_code = "HERITAGE_PARTIAL"
    print(f"\n  VERDICT: {verdict_code}")
    print(f"  Partial heritage protection.")
    print(f"  - Some regions protected, others damaged")
else:
    verdict_code = "HERITAGE_REFUTED"
    print(f"\n  VERDICT: {verdict_code}")
    print(f"  Heritage does NOT protect Qwen2!")
    print(f"  - Alignment damages antifragility")
    print(f"  - Unexpected result!")

results['verdict'] = {
    'code': verdict_code,
    'heritage_score': float(mean_heritage),
    'family': MODEL_CONFIG['family'],
    'architecture': MODEL_CONFIG['architecture'],
    'rho': MODEL_CONFIG['rho'],
    'num_seeds': len(SEEDS)
}

print(f"\n{'='*70}")

In [ ]:
# Cell 9: Visualization

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

colors = {'early': '#3498db', 'middle': '#2ecc71', 'late': '#e74c3c', 'all': '#9b59b6'}

# Plot 1: Base Fragility by Region
ax1 = axes[0, 0]
regions = list(LAYER_RANGES.keys())
base_frags = [results['base']['aggregated'][r]['mean_fragility'] for r in regions]
base_stds = [results['base']['aggregated'][r]['std_fragility'] for r in regions]

bars = ax1.bar(regions, base_frags, yerr=base_stds, capsize=5, 
               color=[colors[r] for r in regions], alpha=0.7)
ax1.axhline(y=0, color='black', linestyle='--')
ax1.set_ylabel('Fragility')
ax1.set_title('BASE Model Fragility\n(Negative = Antifragile)')

# Plot 2: Instruct Fragility by Region
ax2 = axes[0, 1]
inst_frags = [results['instruct']['aggregated'][r]['mean_fragility'] for r in regions]
inst_stds = [results['instruct']['aggregated'][r]['std_fragility'] for r in regions]

bars = ax2.bar(regions, inst_frags, yerr=inst_stds, capsize=5,
               color=[colors[r] for r in regions], alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='--')
ax2.set_ylabel('Fragility')
ax2.set_title('INSTRUCT Model Fragility\n(Negative = Antifragile)')

# Plot 3: Comparison (Base vs Instruct)
ax3 = axes[1, 0]
x = np.arange(len(regions))
width = 0.35

ax3.bar(x - width/2, base_frags, width, label='Base', color='blue', alpha=0.7)
ax3.bar(x + width/2, inst_frags, width, label='Instruct', color='orange', alpha=0.7)
ax3.axhline(y=0, color='black', linestyle='--')
ax3.set_xticks(x)
ax3.set_xticklabels(regions)
ax3.set_ylabel('Fragility')
ax3.set_title('Base vs Instruct Comparison')
ax3.legend()

# Plot 4: Delta (Heritage Damage)
ax4 = axes[1, 1]
deltas = [inst_frags[i] - base_frags[i] for i in range(len(regions))]

bar_colors = ['green' if d < 0.05 else 'orange' if d < 0.15 else 'red' for d in deltas]
ax4.bar(regions, deltas, color=bar_colors, alpha=0.7)
ax4.axhline(y=0, color='black', linestyle='--')
ax4.axhline(y=0.05, color='orange', linestyle=':', label='Partial threshold')
ax4.axhline(y=0.15, color='red', linestyle=':', label='Damage threshold')
ax4.set_ylabel('Δ Fragility (Instruct - Base)')
ax4.set_title('Heritage Damage\n(Green = Protected, Red = Damaged)')
ax4.legend()

plt.suptitle(f'E04-Qwen Twin Test: {verdict_code}\nHeritage Score: {mean_heritage:.2f}/1.0', 
             fontsize=14, fontweight='bold')
plt.tight_layout()

fig_path = f'figures/E04_qwen_twin_{TIMESTAMP}.png'
plt.savefig(fig_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFigure saved: {fig_path}")

In [ ]:
# Cell 10: Save Results (E11-v3 with methodology block)

def convert_to_native(obj):
    if isinstance(obj, dict):
        return {k: convert_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native(v) for v in obj]
    elif isinstance(obj, tuple):
        return tuple(convert_to_native(v) for v in obj)
    elif isinstance(obj, (np.bool_, np.integer)):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

filename = f'results/E04_qwen_twin_{TIMESTAMP}.json'

output = {
    'experiment': 'E04-Qwen-Twin',
    'purpose': 'Heritage > Scale - 3rd Family Validation',
    'timestamp': TIMESTAMP,
    'base_model': BASE_MODEL,
    'instruct_model': INSTRUCT_MODEL,
    'model_config': MODEL_CONFIG,
    'layer_ranges': {k: list(v) for k, v in LAYER_RANGES.items()},
    'noise_levels': NOISE_LEVELS,
    'metric': 'repetition-score (not SI)',
    # E11-v3 Methodology Block
    'methodology': {
        'standard': 'E11-v3',
        'seeds': SEEDS,
        'max_length': MAX_LENGTH,
        'dtype': str(DTYPE),
        'prompt_md5': ACTUAL_MD5,
        'prompt_md5_verified': PROMPTS_VERIFIED,
        'num_prompts': len(STANDARD_PROMPTS),
        'prompt_set': 'Standard-10 v3',
        'quantization': 'bfloat16'
    },
    'results': convert_to_native(results)
}

with open(filename, 'w') as f:
    json.dump(output, f, indent=2)

print(f"Results saved: {filename}")

print(f"\n📋 E11-v3 Compliance:")
print(f"   Seeds: {SEEDS} ✓")
print(f"   dtype: {DTYPE} ✓")
print(f"   MD5: {ACTUAL_MD5} {'✓' if PROMPTS_VERIFIED else '✗'}")
print(f"   MAX_LENGTH: {MAX_LENGTH} ✓")

try:
    from google.colab import files
    files.download(filename)
    files.download(fig_path)
except:
    pass

---

## Summary

### E04-Qwen: Twin Test (Heritage > Scale)

**Purpose:** Validate B3 claim with 3rd family (Qwen2)

**Method:**
1. Load both Base and Instruct models
2. Measure fragility response to noise injection
3. Compare: Does alignment damage antifragility?

**Verdict Criteria:**

| Outcome | Condition | Implication |
|---------|-----------|-------------|
| HERITAGE_CONFIRMED | Δ < 0.05 across regions | Heritage protects |
| HERITAGE_PARTIAL | Mixed results | Partial protection |
| HERITAGE_REFUTED | Δ > 0.15 across regions | Heritage doesn't protect |

**Expected Result:**
- Qwen2 should maintain antifragility after alignment
- This confirms Heritage > Scale for Qwen2 family
- B3 claim upgraded to 3-family validation

---

*Paper 4: Behavioral Sink Dynamics*  
*E04-Qwen: Twin Test*

In [ ]:
# Cell 12: Auto-Download Results

import glob
import shutil

def auto_download_results():
    try:
        from google.colab import files
    except ImportError:
        print('Not in Colab - skipping auto-download')
        return
    
    print('=' * 60)
    print('AUTO-DOWNLOADING RESULTS...')
    print('=' * 60)
    
    json_files = glob.glob('results/*.json') + glob.glob('figures/*.json')
    png_files = glob.glob('results/*.png') + glob.glob('figures/*.png')
    all_files = json_files + png_files
    
    if not all_files:
        print('WARNING: No result files found!')
        return
    
    print(f'Found {len(all_files)} files')
    
    import os
    zip_name = f'E04_qwen_twin_results_{TIMESTAMP}'
    
    os.makedirs('download_package', exist_ok=True)
    for f in all_files:
        shutil.copy(f, 'download_package/')
    
    shutil.make_archive(zip_name, 'zip', 'download_package')
    print(f'Downloading: {zip_name}.zip')
    files.download(f'{zip_name}.zip')
    print('DOWNLOAD COMPLETE!')

auto_download_results()